In [46]:
import tensorflow as tf
import glob
from matplotlib import pyplot as plt
import numpy as np
HEIGHT = 256
WIDTH = 256

In [110]:
def decode_jpg(file_path):
    jpg = tf.io.read_file(file_path)
    jpg = tf.io.decode_jpeg(jpg, channels=3)
    return jpg

def decode_png(file_path):
    png = tf.io.read_file(file_path)
    png = tf.io.decode_png(png, channels=1)
    return png

def process_path(image_fp, mask_fp):
    image = decode_jpg(image_fp)
    mask = decode_png(mask_fp)
    return image, mask
    
def display(display_list):
    plt.figure(figsize=(10,10))
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.imshow(display_list[i], cmap='gray')
        plt.axis('off')
    plt.show()
    
def resize_images(image, mask):
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = image[np.newaxis, :, :,:]
    image = image/255
    image = (image - image.mean())/image.std()
    mask = tf.image.resize(mask, [HEIGHT, WIDTH])
    mask = mask[np.newaxis, :, :,:]
    mask = tf.math.round(mask/255)
    return image, mask
    

In [47]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, LeakyReLU, Conv2D as conv2D, Dropout, Add, UpSampling2D, concatenate


def improved_unet(W, H):
    input_size = (W, H, 3)
    inputs = Input(input_size)
    print(inputs)
    #W*H*16
    conv1 = conv2D(16, 3, activation = tf.keras.layers.LeakyReLU(alpha=0.01), padding = 'same')(inputs)
    cont1 = conv2D(16, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(conv1)
    cont1 = Dropout(0.3)(cont1)
    cont1 = conv2D(16, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(cont1)
    conc1 = Add()([conv1, cont1]) #W*H*16
    # W/2 * H/2 * 32
  
    conv2 = conv2D(32, 3, strides = (2,2), activation = LeakyReLU(alpha=0.01), padding = 'same')(conc1)
    cont2 = conv2D(32, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(conv2)
    cont2 = Dropout(0.3)(cont2)
    cont2 = conv2D(32, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(cont2)
    conc2 = Add()([conv2, cont2]) #W/2*H/2*32

    conv3 = conv2D(64, 3, strides = (2,2), activation = LeakyReLU(alpha=0.01), padding = 'same')(conc2)
    cont3 = conv2D(64, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(conv3)
    cont3 = Dropout(0.3)(cont3)
    cont3 = conv2D(64, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(cont3)
    conc3 = Add()([conv3, cont3]) #W/4*H/4*64

    conv4 = conv2D(128, 3, strides = (2,2), activation = LeakyReLU(alpha=0.01), padding = 'same')(conc3)
    cont4 = conv2D(128, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(conv4)
    cont4 = Dropout(0.3)(cont4)
    cont4 = conv2D(128, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(cont4)
    conc4 = Add()([conv4, cont4]) #W/8*H/8*128
 
    conv5 = conv2D(256, 3, strides = (2,2), activation = LeakyReLU(alpha=0.01), padding = 'same')(conc4)
    cont5 = conv2D(256, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(conv5)
    cont5 = Dropout(0.3)(cont5)
    cont5 = conv2D(256, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(cont5)
    conc5 = Add()([conv5, cont5]) #W/16*H/16*256

    uconv5 = UpSampling2D(size = (2,2))(conc5)
    uconv5 = conv2D(128, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv5)
 
    uconv4 = concatenate([uconv5, conc4])
    lconv4 = conv2D(128, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv4)
    lconv4 = conv2D(128, 1, activation = LeakyReLU(alpha=0.01), padding = 'same')(lconv4)
    uconv4 = UpSampling2D(size = (2,2))(uconv4)
    uconv4 = conv2D(64, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv4)
 
    uconv3 = concatenate([uconv4, conc3])
    lconv3 = conv2D(64, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv3)
    lconv3 = conv2D(64, 1, activation = LeakyReLU(alpha=0.01), padding = 'same')(lconv3)
    sconv3 = conv2D(1, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(lconv3) 
    sconv3 = UpSampling2D(size = (2,2))(sconv3) 
    uconv3 = UpSampling2D(size = (2,2))(lconv3)
    uconv3 = conv2D(32, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv3)

    uconv2 = concatenate([uconv3, conc2])   
    lconv2 = conv2D(32, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv2)
    lconv2 = conv2D(32, 1, activation = LeakyReLU(alpha=0.01), padding = 'same')(lconv2)
    sconv2 = conv2D(1, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(lconv2) 
    sconv2 = Add()([sconv2, sconv3])
    sconv2 = UpSampling2D(size = (2,2))(lconv2)
    uconv2 = UpSampling2D(size = (2,2))(uconv2)
    uconv2 = conv2D(16, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv2)
    
    uconv1 = concatenate([uconv2, conc1])
    uconv1 = conv2D(32, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv1)
    sconv1 = conv2D(1, 3, activation = LeakyReLU(alpha=0.01), padding = 'same')(uconv1) 
    sconv1 = Add()([sconv1, sconv2])
    output = conv2D(1, 1, activation = 'sigmoid')(sconv1)
    model = Model(inputs = inputs, outputs = output)
    return model

                          

    

In [79]:


input_images = sorted(glob.glob("H:\\COMP3710/report/ISIC2018_Task1-2_Training_Data/input/*.jpg"))
output_masks = sorted(glob.glob("H:\\COMP3710/report/ISIC2018_Task1-2_Training_Data/truth/*.png"))
DATASET_SIZE = len(input_images)
train_size = int(DATASET_SIZE * 0.7)
val_size = test_size = int(DATASET_SIZE * 0.15)
train_x = input_images[0:train_size]
train_y = output_masks[0:train_size]
test_x = input_images[train_size:train_size+test_size]
test_y = output_masks[train_size:train_size+test_size]
val_x = input_images[train_size+test_size:]
val_y = output_masks[train_size+test_size:]



def create_dataset(inputs, outputs):
    ds  = tf.data.Dataset.from_tensor_slices((inputs, outputs))
    ds = ds.map(process_path)
    ds = ds.map(resize_images)
    return ds

def plot_image(ds):
    for image, mask in ds.take(1):
#     print(np.unique(image))
        print(np.unique(mask))
        print(image.shape)
        print(mask.shape)
    #image = tf.image.resize(image, (1000, 1512, 1))
        plt.figure()
        plt.imshow(image)
        plt.figure()
        plt.imshow(mask[:,:,0], cmap='gray')


In [101]:
train_ds = create_dataset(train_x, train_y)
test_ds = create_dataset(test_x, test_y)
val_ds = create_dataset(val_x, val_y)
model = improved_unet(HEIGHT, WIDTH)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])



Tensor("input_7:0", shape=(None, 256, 256, 3), dtype=float32)


In [102]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

In [109]:
history = model.fit(train_ds, epochs = 50, steps_per_epoch=50, validation_data = val_ds, validation_steps=20, callbacks=[model_checkpoint_callback])

Train for 50 steps, validate for 20 steps
Epoch 1/50
50/50 [==============================] - 2s 41ms/step - loss: 2.7878 - acc: 0.6462 - val_loss: 0.6253 - val_acc: 0.6796
Epoch 2/50
50/50 [==============================] - 2s 39ms/step - loss: 0.5065 - acc: 0.7925 - val_loss: 0.5112 - val_acc: 0.6858
Epoch 3/50
50/50 [==============================] - 6s 118ms/step - loss: 0.4175 - acc: 0.8240 - val_loss: 0.5644 - val_acc: 0.6743
Epoch 4/50
50/50 [==============================] - 2s 39ms/step - loss: 0.4335 - acc: 0.8169 - val_loss: 0.4552 - val_acc: 0.8020
Epoch 5/50
50/50 [==============================] - 2s 39ms/step - loss: 0.4215 - acc: 0.8123 - val_loss: 0.7319 - val_acc: 0.5256
Epoch 6/50
50/50 [==============================] - 5s 107ms/step - loss: 0.4674 - acc: 0.7774 - val_loss: 0.4744 - val_acc: 0.7637
Epoch 7/50
50/50 [==============================] - 2s 39ms/step - loss: 0.3918 - acc: 0.8546 - val_loss: 0.9163 - val_acc: 0.5986
Epoch 8/50
50/50 [=====================

In [111]:
results = model.evaluate(test_ds)
print(results)

389/389 [==============================] - 28s 73ms/step - loss: 0.3958 - acc: 0.8585
[0.39576435195890214, 0.8585277]


In [112]:
predictions = model.predict(test_ds)

In [146]:
print(predictions.shape)
true_masks = []
for x, y in test_ds:
    true_masks.append(y[0,:,:,:])
true_masks = np.array(true_masks)
print(true_masks.shape)

(389, 256, 256, 1)
(389, 256, 256, 1)


In [147]:
pred_labels = np.round(predictions)
pre_labels = pred_labels.flatten()
coe = 2 * sum(pre_labels == true_masks.flatten())/(2*len(labels))
print(coe)

0.8585278430144401


KeyboardInterrupt: 

In [133]:
round(1.8)

2